**NOTE**: to get this working, you must install the `data-analysis` group dependencies using Poetry:

```shell
poetry install --with=data-analysis
```

You should also run this if you are using git:

```shell
poetry run nbstripout --install
```

This will automatically clear output when committing to git :)

In [ ]:
import json
import sqlite3
from operator import itemgetter

import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score

from data_analysis_utils import agreement_as_label

Ways to index and group data:

In [ ]:
# by each Java source code file (unit):
# TODO: rename to "context"?
BY_UNIT = ["srcml_path", "version"]
# by the PEM variant shown to the rater:
BY_SCENARIO = BY_UNIT + ["variant"]
# by the rater: this uniquely identifies one particular data point:
BY_RATER = BY_SCENARIO + ["rater"]

We'll create a basic dataframe from `answers.sqlite3`. However, the `answers` column will need to be parsed as JSON.

In [ ]:
# Read answers.sqlite3 into a DataFrame
conn = sqlite3.connect("answers.sqlite3")
df = pd.read_sql_query("SELECT * FROM answers", conn)
conn.close()
df

Now time to clean the data — extract it from that JSON column.

Instead of creating a nice schema in `answers.sqlite3`, I decided to defer the job of making nice columns to the data analysis stage. So now we have to parse the answers column as JSON and extract data.


As of 2023-05-02, these are columns:

In [ ]:
json.loads(df["answers"][0])

In [ ]:

df["answers"].apply(json.loads).apply(itemgetter("fix")).value_counts()
df["answers"].apply(json.loads).apply(lambda x: x["fix"]).value_counts()

In [ ]:
def json_to_columns(df):
    json_column = df["answers"].apply(json.loads)
    return df.assign(
        jargon=(
            json_column.apply(itemgetter("jargon")).astype(int)
        ),
        sentence_structure=(
            json_column.apply(itemgetter("sentence_structure")).astype("category")
        ),
        # Explanation should already be a boolean column, that is completely filled in:
        explanation=(
            json_column.apply(itemgetter("explanation"))
        ),
        explanation_correctness=(
            json_column.apply(itemgetter("explanation_correctness")).astype("category")
        ),
        # I wish I had a better name for this column, but it's basically, "if the explanation is MAYBE correct, WHY is it maybe correct?"
        explanation_maybe=(
            json_column.apply(itemgetter("explanation_maybe")).astype("category")
        ),
        fix=(
            json_column.apply(itemgetter("fix")).fillna("N/A").astype("category")
        ),
        fix_correctness=(
            json_column.apply(itemgetter("fix_correctness")).astype("category")
        ),
        additional_errors=(
            json_column.apply(itemgetter("additional_errors")).astype("category")
        ),
        notes=(
            json_column.apply(itemgetter("notes")).astype("string")
        ),
        length=(
            json_column.apply(itemgetter("length")).astype(int)
        ),
    )

def variant_as_categorical(df):
    variant = df["variant"].astype("category")
    assert len(variant.cat.categories) == 4
    return df


def rater_as_categorical(df):
    rater = df["rater"].astype("category")
    assert len(rater.cat.categories) == 3
    return df.assign(rater=rater)


def set_empty_notes_to_na(df):
    "Notes that are empty strings should just be missing values"
    return df.assign(notes=df["notes"].replace("", pd.NA))


expanded_df = df\
    .pipe(json_to_columns)\
    .pipe(variant_as_categorical)\
    .pipe(rater_as_categorical)\
    .pipe(set_empty_notes_to_na)\
    .drop(columns=["answers"])

expanded_df.sample(5)

In [ ]:
expanded_df.info()

What are the levels of the categorical variables?

In [ ]:
# Get the levels for each categorical variable in df
def get_levels(df, skip=()):
    for column in df.select_dtypes("category").columns:
        if column in skip:
            continue
        print(df[column].value_counts())
        print(set(df[column].cat.categories))
        print()


get_levels(expanded_df, skip=["variant", "rater"])

For further data analysis, it's useful to think of some of the categorical responses as being ordinal. It will also smooth over inter-rater reliability (you could argue that it's cooking the books), since if two raters answer "yes" and "maybe", that's more agreement than two raters saying "yes" and "no".

There are also some columns that can be made binary, like "does it provide a correct fix?" or "is the explanation definitely correct?"

In [ ]:
sentence_structure_levels = {'unclear': 0, 'could-be-clearer': 1, 'clear': 2}
explanation_correctness_levels = {'no': 0, 'maybe': 1, 'yes': 2}
fix_levels = {'no': 0, 'implicit-suggestion': 1, 'generic': 2, 'hint': 3, 'confident': 4}
fix_correctness_levels = {'no': 0, 'maybe': 1, 'yes': 2}
additional_errors_levels = {'no': 0, 'maybe': 1, 'yes': 2}

def add_ordinal_columns(df):
    return df.assign(
        sentence_structure_ordinal=df["sentence_structure"].map(sentence_structure_levels),
        explanation_correctness_ordinal=df["explanation_correctness"].map(explanation_correctness_levels),
        fix_ordinal=df["fix"].map(fix_levels),
        fix_correctness_ordinal=df["fix_correctness"].map(fix_correctness_levels),
        additional_errors_ordinal=df["additional_errors"].map(additional_errors_levels),
    )

def add_binary_columns(df):
    return df.assign(
        # Is the sentence structure clear?
        sentence_structure_binary=df["sentence_structure"].map({"unclear": 0, "could-be-clearer": 0, "clear": 1}),
        # Can the explanation be reasonably considered to be correct?
        explanation_correctness_binary=df["explanation_correctness"].map({"no": 0, "maybe": 1, "yes": 1}),
        # Is **any** kind of fix suggested?
        fix_binary=df["fix"].map({'no': 0, 'implicit-suggestion': 1, 'generic': 1, 'hint': 1, 'confident': 1}),
        # Is the fix DEFINITELY correct?
        fix_correctness_binary=df["fix_correctness"].map({"no": 0, "maybe": 0, "yes": 1}),
        additional_errors_binary=df["additional_errors"].map({"no": 0, "maybe": 1, "yes": 1}),
    )


full_df = expanded_df\
    .pipe(add_ordinal_columns)\
    .pipe(add_binary_columns)

full_df.sample(5)

Every rating **must** state whether an explanation was provided or not. Assert this here:

In [ ]:
assert full_df["explanation"].isna().sum() == 0

The length should be the equal regardless of the rater, so let's assert that here and get a table we can join with later.

In [ ]:
def assert_lengths(df):
    "Ensure that each rater has seen a message with the same length."
    assert (df["length"]["min"] == df["length"]["max"]).all()
    return df


def as_sql_style_table(df):
    return pd.DataFrame({
        "srcml_path": df["srcml_path"],
        "version": df["version"],
        "variant": df["variant"].astype("category"),
        "length": df["length"]["min"],
    })

# For some bizarre reason, the fact that variant is a categorical
# variable completely changes the behavior of groupby.
# I have no idea why.
# Instead of fighting it, I convert variant to a string column,
# perform the groupby, then convert it BACK to a categorical column.
message_lengths = full_df\
    .pipe(lambda df: df.assign(variant=df["variant"].astype("string")))\
    .groupby(BY_SCENARIO, as_index=False)\
    .agg({"length": ["min", "max"]})\
    .pipe(assert_lengths)\
    .pipe(as_sql_style_table)

message_lengths.sample(5)

Assert that all scenarios were answered by at least two raters:

In [ ]:
two_ratings = full_df.groupby(["srcml_path", "version"]).filter(lambda x: len(x["rater"].unique()) >= 2)
assert len(two_ratings) == len(full_df), "Had fewer than two ratings for some scenarios"

In [ ]:
full_df.columns

In [ ]:
FIXED_COLUMNS = {'srcml_path', 'version', 'variant', 'rater', 'length'}

COLUMNS_TO_RENAME = [
    column for column in full_df.columns
    if column not in FIXED_COLUMNS
]

# TODO: make a hierarchical index?
def get_ratings(df, rater):
    return (
        df[df["rater"] == rater]
        .set_index(BY_SCENARIO)
        # We don't need the rater column anymore
        # and earlier we confirmed that the length column is identical
        .drop(columns=["rater", "length"])
        # We have to rename the columns here because the three-way merge will fail to recognize identical columns
        # otherwise:
        .rename(columns={col: f"{col}_{rater}" for col in COLUMNS_TO_RENAME})
    )

# Get the explanation ratings from prajish and eddie:
eddie_ratings = get_ratings(two_ratings, "eddie")
prajish_ratings = get_ratings(two_ratings, "prajish")
brett_ratings = get_ratings(two_ratings, "brett")

assert len(eddie_ratings) == len(prajish_ratings) == len(brett_ratings)
assert len(eddie_ratings) == len(full_df) / full_df["rater"].nunique()

In [ ]:
ratings = prajish_ratings.join([eddie_ratings, brett_ratings], how="outer")

ratings.sample(3)

In [ ]:
ratings.info()

In [ ]:
eddie_and_prajish_agreement = ratings[ratings["explanation_eddie"].notna() & ratings["explanation_prajish"].notna()]
eddie_and_prajish_agreement

In [ ]:
# TODO: perhaps Scott's pi is a better measure of agreement for this data:
# See: https://stats.stackexchange.com/a/525640

eddie_and_prajish_agreement = ratings[ratings["explanation_eddie"].notna() & ratings["explanation_prajish"].notna()]

kappa = cohen_kappa_score(eddie_and_prajish_agreement["explanation_eddie"].astype(bool), eddie_and_prajish_agreement["explanation_prajish"].astype(bool))
kappa, agreement_as_label(kappa)

In [ ]:
def group_two_raters(rater1, rater2):
    "Returns a DataFrame with only the rows where both raters have rated the scenario"
    return ratings[ratings[f"explanation_{rater1}"].notna() & ratings[f"explanation_{rater2}"].notna()]

def kappa_for_two_raters(column, rater1, rater2):
    "Returns agreement for two raters on a certain column"
    agreement = group_two_raters(rater1, rater2)
    return cohen_kappa_score(agreement[f"{column}_{rater1}"].astype("category"), agreement[f"{column}_{rater2}"].astype("category"))

def kappa_for_two_raters_with_label(column, rater1, rater2):
    kappa = kappa_for_two_raters(column, rater1, rater2)
    return kappa, agreement_as_label(kappa)

In [ ]:
kappa_for_two_raters_with_label("explanation", "eddie", "prajish")

In [ ]:
kappa_for_two_raters_with_label("explanation", "eddie", "brett")

In [ ]:
kappa_for_two_raters_with_label("explanation", "prajish", "brett")

In [ ]:
kappa_for_two_raters_with_label("fix", "eddie", "prajish")

In [ ]:
kappa_for_two_raters_with_label("fix", "eddie", "brett")

In [ ]:
kappa_for_two_raters_with_label("fix", "prajish", "brett")

In [ ]:
eddie = ratings["explanation_eddie"]
brett = ratings["explanation_brett"]
prajish = ratings["explanation_prajish"]

rater1 = prajish.combine_first(brett)
rater2 = eddie.combine_first(brett)

In [ ]:
kappa = cohen_kappa_score(rater1.astype("category"), rater2.astype("category"))
kappa, agreement_as_label(kappa)

In [ ]:
def coallesce_agreement(column):
    eddie = ratings[f"{column}_eddie"]
    brett = ratings[f"{column}_brett"]
    prajish = ratings[f"{column}_prajish"]

    # For null coallescing to work properly, the alternative option must be the same for both columns
    # In other words, Brett must ALWAYS be last:
    rater1 = prajish.combine_first(brett).astype("category")
    rater2 = eddie.combine_first(brett).astype("category")

    assert rater1.isna().sum() == rater2.isna().sum() == 0

    kappa = cohen_kappa_score(rater1, rater2)
    return kappa, agreement_as_label(kappa)

In [ ]:
coallesce_agreement("explanation")

In [ ]:
coallesce_agreement("explanation_correctness")

In [ ]:
coallesce_agreement("fix")

In [ ]:
coallesce_agreement("fix_correctness")

In [ ]:
#full_df\
#    .assign(explanation=full_df["explanation"].astype(int))\
#    .pivot(index=["srcml_path", "version", "variant"], columns="rater", values=["explanation", "fix", "notes"]).to_excel("full_df.xlsx")